In [46]:
import numpy as np
import plotly.graph_objects as go
import csv
from scipy import optimize
from scipy import stats
import sys,os
from sklearn.linear_model import LinearRegression

# Read data files

In [47]:
def read_data(file_path):
    with open(file_path) as f:
        reader = csv.reader(f)
        data = []
        for column in reader:
            if len(column) == 3:
                column  = column[:2]
                column[0] = column[0].replace('(', '').replace("'" , '')
                # column[2] = column[2].replace(')', '').replace("'" , '')
                column[1] = column[1].replace("'" , '')
                column[0] = float(column[0])
                column[1] = float(column[1])
            else:
                column[0] = float(column[0])
                column[1] = float(column[1])
            data.append(column)
        data = np.array(data)
    return data

# 2D xy projection position plot

In [48]:
def plot_pos(arrays,energies,particle):
    # Create the output directory if it doesn't already exist
    output_dir = "../media/plots/"+f"{particle}"+"/pos/"
    os.makedirs(output_dir, exist_ok=True)

    for i, array in enumerate(arrays):
        energy_str = str(energies[i]).replace('.', '_')
        fig = go.Figure(data=go.Scatter(x=array[:,0], y=array[:,1], mode='markers'))
        fig.update_layout(title=f"{energy_str}"+ 'G', xaxis_title='x', yaxis_title='y')
        fig.update_layout(
            width=500,
            height=500,
            margin=dict(
                l=50,
                r=50,
                b=100,
                t=100,
                pad=4
            ),
            xaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1
            ),
            yaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1
            )
        )
        # fig.show()
        fig.write_image("../media/plots/"+f"{particle}"+"/pos/"+f"{energies[i]}"+ 'G'+".png")


# Calculate physical quantities

In [49]:
def momentum(E, m):
    E = float(E)
    return np.sqrt(E**2 - m**2)

In [50]:
def get_beta(energy, momentum):
    beta = []
    for i in range(len(energy)):
        beta.append(momentum[i]/energy[i])
    return np.array(beta)

In [51]:
#find the slope of plot
def get_slope(x, y):
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
    return slope


# Fit circle for radius calculation

In [52]:
def fit_circle(data):
    x, y = data[:,0], data[:,1]
    x_m, y_m = np.mean(x), np.mean(y)

    def calc_R(xc, yc):
        """ calculate the distance of each 2D points from the center (xc, yc) """
        return np.sqrt((x-xc)**2 + (y-yc)**2)

    def f(c):
        """ calculate the algebraic distance between the data points and the mean circle centered at c=(xc, yc) """
        Ri = calc_R(*c)
        return Ri - Ri.mean()

    center_estimate = x_m, y_m
    center, _ = optimize.leastsq(f, center_estimate)

    xc, yc = center
    Ri       = calc_R(*center)
    R       = Ri.mean()
    return xc, yc, R

## Plot functions

In [53]:
def plot_circle(xc, yc, radii, arrays, energies, particle):
    # Create the output directory if it doesn't already exist
    output_dir = "../media/plots/"+f"{particle}"+"/fit/"
    os.makedirs(output_dir, exist_ok=True)
    for i, array in enumerate(arrays):
        # Create the output directory if it doesn't already exist
        output_dir = "../media/plots/1.1_pion-/fit/"
        os.makedirs(output_dir, exist_ok=True)

        energy_str = str(energies[i]).replace('.', '_')
        theta_fit = np.linspace(0, 2*np.pi, 180)
        x_fit = xc[i] + (radii[i])*np.cos(theta_fit)
        y_fit = yc[i] + (radii[i])*np.sin(theta_fit)
        fig = go.Figure(data=go.Scatter(x=x_fit, y=y_fit, mode='lines', marker=dict(color='blue', opacity=0.8, symbol='square', size=3)))
        fig.add_trace(go.Scatter(x=array[:,0], y=array[:,1], mode='markers', marker=dict(color='violet', size=3, opacity=0.8)))
        fig.update_layout(title=f"{energy_str}"+ 'G', xaxis_title='y', yaxis_title='z')

        fig.add_trace(go.Histogram(
                            x=array[:,0],
                            y=array[:,1],
                            histfunc='density',
                            marker=dict(color='darkgreen', size=3, opacity=0.8)
                        )
                    )

        fig.update_layout(
            plot_bgcolor='#F5F5F5',
            width=800,
            height=800,
            margin=dict(
                l=50,
                r=50,
                b=100,
                t=100,
                pad=4
            ),
            xaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1,
                scaleanchor="Posiciones en x",
                range=[-200, 200],
            ),
            yaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1,
                scaleanchor="Posiciones en y",
                range=[-200, 200]
            )
        )
        # fig.show()
        fig.write_image("../media/plots/"+f"{particle}"+"/fit/"+f"{energies[i]}"+ 'G'+".png")



In [54]:
def plot_cos_beta(cos,beta_1, particle):
    # Create the output directory if it doesn't already exist
    output_dir = "../media/plots/beta/"
    os.makedirs(output_dir, exist_ok=True)
    # output_dir = "../media/plots/1.1_pion-/beta/"
    fig = go.Figure(data=go.Scatter(x=cos, y=beta_1, mode='markers', marker=dict(color='darkgreen', opacity=0.8)))
    fig.update_layout(title='Cherenkov angle vs beta', xaxis_title='beta', yaxis_title='cherenkov angle', width=500, height=500)
    fig.write_image("../media/plots/beta/"+f"{particle}"+".png")
    # fig.show()

# Calculate Cherenkov cone aperture angle

In [55]:
def plot_momenta_cherenkov_angle(data, particle):
    # Create the output directory if it doesn't already exist
    # output_dir = "../media/plots/1.1_pion-/p/"
    output_dir = "../media/plots/p/"
    os.makedirs(output_dir, exist_ok=True)
    fig = go.Figure(data=go.Scatter(x=data[:,2], y=data[:,3], mode='markers', marker=dict(color='darkgreen', opacity=0.8, size=2)))
    fig.update_layout(title='Cherenkov angle vs momenta', xaxis_title='momenta', yaxis_title='cherenkov angle', width=500, height=500)
    fig.write_image("../media/plots/p/"+f"{particle}"+".png")
    # fig.show()

## Pion - n=1.1

In [65]:
#Read energy in GeV of data files, save to list named energies
energies_pion = []

#Read all pion datasets
for root, dir, files in os.walk('../data/n1.1pion-'):
    for file in files:
        if file.endswith('.csv'):
            energies_pion.append(file.replace('.csv', '').replace('_', '.').replace('G', ''))
energies_pion = np.array(energies_pion, dtype=float)

pion_data = []
for root, dir, files in os.walk('../data/n1.1pion-'):
    for file in files:
        if file.endswith('.csv'):
            data = read_data(os.path.join(root, file))
            pion_data.append(data)

plot_pion_dir = '../media/plots/1.1_pion-/pos/'
plot_pos(pion_data, energies_pion, 'n1.1_pion-')

#Circle fit for pion data
radii_pion = []
xc_pion=[]
yc_pion=[]
momenta_pion = []


#quantities calculated for pion
# #Mass in GeV
for e in range(len(energies_pion)):
    p = momentum(energies_pion[e], 0.13957)
    # print(energies_pion[e])
    # print(p)
    momenta_pion.append(p)
momenta_pion = np.array(momenta_pion)
for array in pion_data:
    radii_pion.append(fit_circle(array)[2])
    xc_pion.append(fit_circle(array)[0])
    yc_pion.append(fit_circle(array)[1])
radii_pion = np.array(radii_pion)
xc_pion = np.array(xc_pion)
yc_pion = np.array(yc_pion)
# print("momenta array", momenta_pion.shape, "energies", energies_pion.shape, "radios ",radii_pion.shape)
# distancia 0.24 m
d = 240 # mm

pion_cherenkov_angle = []
for i, array in enumerate(pion_data):
    theta = np.arctan(radii_pion[i]/d)
    theta_deg = np.degrees(theta)
    pion_cherenkov_angle.append(theta_deg)
pion_cherenkov_ang = np.array(pion_cherenkov_angle)
pion_physdata = np.column_stack((energies_pion, radii_pion, momenta_pion,pion_cherenkov_ang))
# print(processed_data.shape)
pion_directory = '../data/exported_data/1.1_pion-/'
if not os.path.exists(pion_directory):
    os.makedirs(pion_directory)
np.savetxt(pion_directory + 'pion_physdata.dat', pion_physdata, delimiter=',')

plot_circle(xc_pion, yc_pion, radii_pion, pion_data, energies_pion, "n1.1_pion-")

# plot_momenta_cherenkov_angle(pion_physdata, particle="1_1pion")

beta_pion = get_beta(energies_pion, momenta_pion)
cos_theta_pion = np.cos(np.radians(pion_cherenkov_angle))
cos_theta_pion_deg = np.degrees(cos_theta_pion)
beta_1_pion = 1/beta_pion
#
# print(pion_cherenkov_angle)
# print(cos_theta_pion)
# print(beta_1_pion)

# plot_cos_beta(cos_theta_pion, beta_1_pion, "n_1_1_pion")

slope = get_slope(beta_1_pion,cos_theta_pion)
print("n", (1/slope))
print("slope", slope)

n 1.166623726442029
slope 0.8571744062241917


## Kaon - n=1.1

In [66]:
energies_kaon = []

#Read all kaon datasets
for root, dir, files in os.walk('../data/n1.1kaon-'):
    for file in files:
        if file.endswith('.csv'):
            energies_kaon.append(file.replace('.csv', '').replace('_', '.').replace('G', ''))
energies_kaon = np.array(energies_kaon, dtype=float)

kaon_data = []

for root, dir, files in os.walk('../data/n1.1kaon-'):
    for file in files:
        if file.endswith('.csv'):
            data = read_data(os.path.join(root, file))
            kaon_data.append(data)


plot_kaon_dir = '../media/plots/1.1_kaon-/pos/'
# plot_pos(kaon_data, energies_kaon,'n1.1_kaon-')

#Circle fit for kaon data
radii_kaon = []
xc_kaon=[]
yc_kaon=[]
momenta_kaon = []


#quantities calculated for kaon
#Mass in GeV
for e in range(len(energies_kaon)):
    p = momentum(energies_kaon[e], 0.493677)
    # print(energies_kaon[e])
    # print(p)
    momenta_kaon.append(p)
momenta_kaon = np.array(momenta_kaon)
for array in kaon_data:
    radii_kaon.append(fit_circle(array)[2])
    xc_kaon.append(fit_circle(array)[0])
    yc_kaon.append(fit_circle(array)[1])
radii_kaon = np.array(radii_kaon)
xc_kaon = np.array(xc_kaon)
yc_kaon = np.array(yc_kaon)
# print("momenta array", momenta_kaon.shape, "energies", energies_kaon.shape, "radios ",radii_kaon.shape)
# distancia 0.24 m
d = 240 # mm
kaon_cherenkov_angle = []
for i, array in enumerate(kaon_data):
    theta = np.arctan(radii_kaon[i]/d)
    theta = np.degrees(theta)
    kaon_cherenkov_angle.append(theta)
kaon_cherenkov_ang = np.array(kaon_cherenkov_angle)
kaon_physdata = np.column_stack((energies_kaon, radii_kaon, momenta_kaon,kaon_cherenkov_ang))
print(kaon_physdata.shape)
kaon_directory = '../data/exported_data/1.1_kaon-/'
if not os.path.exists(kaon_directory):
    os.makedirs(kaon_directory)
np.savetxt(kaon_directory + 'kaon_physdata.dat', kaon_physdata, delimiter=',')

plot_circle(xc_kaon, yc_kaon, radii_kaon, kaon_data, energies_kaon,"n1.1_kaon-")

# plot_momenta_cherenkov_angle(kaon_physdata, particle="1.1_kaon-")

beta_kaon = get_beta(energies_kaon, momenta_kaon)
cos_theta_kaon = np.cos(np.radians(kaon_cherenkov_angle))
beta_1_kaon = 1/beta_kaon
print(cos_theta_kaon)

# plot_cos_beta(cos_theta_kaon, beta_1_kaon)
kaon_slope = get_slope(beta_1_kaon,cos_theta_kaon)
print("slope", (1/kaon_slope))

(21, 4)
[0.89888864 0.89998851 0.90083862 0.90243817 0.90338613 0.91145888
 0.91772548 0.93196213 0.93569992 0.94039303 0.94607884 0.9522005
 0.9700354  0.98148142 0.89935947 0.90528908 0.90799587 0.92889362
 0.96005723 0.99624203 0.89918991]
slope 1.16318049778449


## Proton - n=1.1

In [72]:
energies_proton = []

#Read all proton datasets
for root, dir, files in os.walk('../data/n1.1proton'):
    for file in files:
        if file.endswith('.csv'):
            energies_proton.append(file.replace('.csv', '').replace('_', '.').replace('G', ''))
energies_proton = np.array(energies_proton, dtype=float)


proton_data = []
for root, dir, files in os.walk('../data/n1.1proton'):
    for file in files:
        if file.endswith('.csv'):
            data = read_data(os.path.join(root, file))
            proton_data.append(data)


plot_proton_dir = '../media/plots/1.1_proton/pos/'

plot_pos(proton_data, energies_proton, 'n1.1_proton')

#Circle fit for proton data
radii_proton = []
xc_proton=[]
yc_proton=[]
momenta_proton = []



#quantities calculated for proton
#Mass in GeV
for e in range(len(energies_proton)):
    p = momentum(energies_proton[e], 0.938272)
    # print(energies_proton[e])
    # print(p)
    momenta_proton.append(p)
momenta_proton = np.array(momenta_proton)
for array in proton_data:
    radii_proton.append(fit_circle(array)[2])
    xc_proton.append(fit_circle(array)[0])
    yc_proton.append(fit_circle(array)[1])
radii_proton = np.array(radii_proton)
xc_proton = np.array(xc_proton)
yc_proton = np.array(yc_proton)
# print("momenta array", momenta_proton.shape, "energies", energies_proton.shape, "radios ",radii_proton.shape)
# distancia 0.24 m
d = 249 # mm
proton_cherenkov_angle = []
for i, array in enumerate(proton_data):
    theta = np.arctan(radii_proton[i]/d)
    theta = np.degrees(theta)
    proton_cherenkov_angle.append(theta)
proton_cherenkov_ang = np.array(proton_cherenkov_angle)
proton_physdata = np.column_stack((energies_proton, radii_proton, momenta_proton,proton_cherenkov_ang))
print(proton_physdata.shape)
proton_directory = '../data/exported_data/1.1_proton-/'
if not os.path.exists(proton_directory):
    os.makedirs(proton_directory)
np.savetxt(proton_directory + 'proton_physdata.dat', proton_physdata, delimiter=',')

plot_circle(xc_proton, yc_proton, radii_proton, proton_data, energies_proton,"n1.1_proton")
plot_momenta_cherenkov_angle(proton_physdata, "1.1_proton-")
beta_proton = get_beta(energies_proton, momenta_proton)
cos_theta_proton = np.cos(np.radians(proton_cherenkov_ang))
beta_1_proton = (1/(beta_proton))

plot_cos_beta(beta_1_proton,cos_theta_proton,"n1.1_proton-")

proton_slope = get_slope (beta_1_proton,cos_theta_proton)
print("n", (1/proton_slope))
print("slope", slope)

(27, 4)
n 1.252740112962141
slope 0.8571744062241917
